# Encoders mini-project:
## Implementation of a numerical sequence generator

### I. Introduction

As seen in the explanation.ipnyb notebook, enven though decoders as implemented in this repository were introduced as part of a bigger architecture - the *transformer* architecture - they can be used as a standalone architecture for sequences generation. 

In this notebook, we will implement a simple sequence generator and make different tests and observation to illustrate what was said in the explanations. 

### II. Implementation of the model

First, let's import the code from model.py. The containt of this file if precisely what was done in the explanations notebook:

In [22]:
from model import SelfAttention, TransformerBlock, StandaloneDecoderBlock, StandaloneDecoder
import torch
import torch.nn as nn
import torch.nn.functional as F

### III. Configuration

In [23]:
BATCH_SIZE = 64
BLOCK_SIZE = 256
NUM_EPOCHS = 5
EMBED_SIZE = 256
NUM_LAYERS = 4
NUM_HEADS = 4
FORWARD_EXPANSION = 4
LEARNING_RATE = 5e-4
DROPOUT = 0.2
MAX_LENGTH=30
PAD_TOKEN_ID=11


### IV. Creation of the dataset

For this mini-project, we will use the tiny Shakespear dataset, avaiable at https://www.kaggle.com/datasets/thedevastator/the-bards-best-a-character-modeling-dataset

In [24]:
from torch.utils.data import Dataset
import random as rd
import numpy as np
import string

class ShakespearDataset(Dataset):
    def __init__(self,
                 folder="data/train.csv",
                 block_size=256,
                 stride=1):
        super().__init__()

        # Read file
        with open(folder, 'r', encoding='UTF-8') as f:
            data = f.read()

        # Convert to lowercase:
        data = data.lower()

        # Make vocabulary: 
        self.vocab = list(string.ascii_lowercase) + [':', ',', '\'', ';', '-', '.', '?', '!', '(', ')', ' ', '\"', "&", "\n"] # + ['<PAD>', '<SOS>', '<EOS>'] ne sera pas utile pour ce projet

        # Clean text from eventual characters that are not in the vocabulary:
        cleaned_data = ""
        for char in data:
            if char in self.vocab:
                cleaned_data += char

            
        self.vocab2idx = {token: i for i, token in enumerate(self.vocab)}
        self.idx2vocab = {i: token for token, i in self.vocab2idx.items()}

        self.block_size = block_size
        self.stride = stride

        # Encode data:
        self.ids = torch.tensor([self.vocab2idx[ch] for ch in cleaned_data], dtype=torch.long)

        self.num_samples = max(0, (len(self.ids) - (self.block_size + 1) + self.stride) // self.stride)
        
    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        start = idx * self.stride
        end = start + self.block_size
        x = self.ids[start:end]       
        y = self.ids[start+1:end+1]           
        
        return x, y
    
    def encode(self, s: str) -> torch.Tensor:
        return torch.tensor([self.vocab2idx[ch] for ch in s if ch in self.vocab2idx], dtype=torch.long)

    def decode(self, ids) -> str:
        if isinstance(ids, torch.Tensor):
            ids = ids.tolist()
        return "".join(self.idx2vocab[int(i)] for i in ids)


We will also need to implement the associated dataloader. We will only use a subset for faster trainings

In [25]:
from torch.utils.data import DataLoader
from torch.utils.data import Subset, RandomSampler

train_dataset = ShakespearDataset(
    folder="data/train.csv",
    block_size=BLOCK_SIZE
)

test_dataset = ShakespearDataset(
    folder="data/test.csv",
    block_size=BLOCK_SIZE
)

N_TRAIN = 50000
N_EVAL = 10000
train_subset = Subset(train_dataset, list(range(N_TRAIN)))
test_subset = Subset(test_dataset, list(range(N_EVAL)))

train_loader = DataLoader(
    train_subset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    drop_last=True, # Drop last batch if number of samples not dividible by batch_size
)

test_loader = DataLoader(
    test_subset, 
    batch_size=BATCH_SIZE
)

### V. Training

In [26]:
import torch.optim as optim

if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon GPU
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"USING DEVICE: {device}")

model = StandaloneDecoder(
    trg_vocab_size=len(train_dataset.vocab),
    embed_size=EMBED_SIZE,
    num_layers=NUM_LAYERS,
    num_heads=4,
    forward_expansion=FORWARD_EXPANSION,
    dropout=DROPOUT,
    device=device,
    max_length=BLOCK_SIZE
)

model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss() 


USING DEVICE: mps


In [27]:
x, y = next(iter(train_loader))     # x, y: [B, T]
x, y = x.to(device), y.to(device)
logits = model(x)                   # attendu: [B, T, V]
print("logits:", logits.shape, "targets:", y.shape)

logits: torch.Size([64, 256, 40]) targets: torch.Size([64, 256])


In [28]:
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm

# -------------------------------------------------
# Helpers de vérification
# -------------------------------------------------
def check_model_device(model, device):
    param_devices = {p.device.type for p in model.parameters()}
    buffer_devices = {b.device.type for b in model.buffers()}
    if param_devices:
        assert all(d == device.type for d in param_devices), \
            f"Params not all on {device.type}: {param_devices}"
    if buffer_devices:
        assert all(d == device.type for d in buffer_devices), \
            f"Buffers not all on {device.type}: {buffer_devices}"
    print(f"✓ model devices -> params: {sorted(param_devices) or 'none'} | buffers: {sorted(buffer_devices) or 'none'}")

def check_batch(x, y, device, V=None):
    assert x.ndim == 2 and y.ndim == 2, f"Bad dims: x{tuple(x.shape)} y{tuple(y.shape)}"
    assert x.shape == y.shape, f"x.shape {x.shape} != y.shape {y.shape}"
    # Compare le type de device, pas l'objet exact
    assert x.device.type == device.type and y.device.type == device.type, \
        f"Batch not on {device.type}: x{x.device} y{y.device}"
    assert y.dtype == torch.long, f"Targets must be int64/long, got {y.dtype}"
    assert x.dtype in (torch.long, torch.int64, torch.int32), f"Inputs should be ints, got {x.dtype}"
    if V is not None:
        ymax = int(y.max().item())
        assert ymax < V, f"Target id {ymax} >= vocab size {V}"
    print(f"✓ batch ok -> {tuple(x.shape)} on {x.device} (expected {device}) dtypes: x={x.dtype}, y={y.dtype}")

def check_logits(logits, y, device):
    assert logits.device.type == device.type, f"logits on {logits.device}, expected {device}"
    assert logits.ndim == 3, f"logits must be [B,T,V], got {tuple(logits.shape)}"
    B, T, V = logits.shape
    assert (B, T) == tuple(y.shape), f"logits [B,T,V]={B,T,V} incompatible with targets {tuple(y.shape)}"
    assert logits.dtype in (torch.float16, torch.float32, torch.bfloat16), f"Unexpected dtype for logits: {logits.dtype}"
    print(f"✓ logits ok -> [B,T,V]={B,T,V} on {logits.device}, dtype={logits.dtype}")
    return B, T, V


def count_params(model):
    n = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return n, trainable

# -------------------------------------------------
# Device info (prints utiles)
# -------------------------------------------------
if torch.cuda.is_available():
    dev_name = torch.cuda.get_device_name()
    print(f"CUDA visible | device: {torch.cuda.current_device()} -> {dev_name}")
elif torch.backends.mps.is_available():
    print("MPS visible (Apple Silicon)")
else:
    print("CPU only")

# Déplacement modèle + opti/critère
model = model.to(device)
check_model_device(model, device)

total_params, trainable_params = count_params(model)
print(f"Paramètres: {total_params/1e6:.2f}M (dont entraînables: {trainable_params/1e6:.2f}M)")

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.95), weight_decay=0.1)
criterion = nn.CrossEntropyLoss()

# -------------------------------------------------
# Epoch loop avec vérifs
# -------------------------------------------------
def run_epoch(loader, train=True, max_batches=None, log_interval=50, desc=None, vocab_size=None):
    model.train(mode=train)
    total_loss, total_tokens = 0.0, 0
    ema_loss = None
    start_time = time.time()

    if desc is None:
        desc = "train" if train else "eval"

    pbar = tqdm(enumerate(loader), total=len(loader), desc=desc, leave=False)
    for b, (x, y) in pbar:
        if max_batches is not None and b >= max_batches:
            break

        # Move to device
        x = x.to(device, non_blocking=(device.type == "cuda"))
        y = y.to(device, non_blocking=(device.type == "cuda"))

        # Vérifs batch
        if b == 0:  # afficher une fois par epoch
            check_batch(x, y, device, V=vocab_size)

        if train:
            optimizer.zero_grad(set_to_none=True)

        # forward
        logits = model(x)  # [B, T, V]

        # Vérifs logits
        if b == 0:
            B, T, V = check_logits(logits, y, device)
        else:
            B, T, V = logits.shape

        loss = criterion(logits.reshape(B*T, V), y.reshape(B*T))

        # backward + step
        grad_norm = None
        if train:
            loss.backward()
            grad_norm = clip_grad_norm_(model.parameters(), max_norm=1.0).item()
            optimizer.step()

        # stats globales
        tokens = B * T
        total_loss += loss.item() * tokens
        total_tokens += tokens

        # EMA pour un affichage plus stable
        ema_loss = loss.item() if ema_loss is None else 0.9 * ema_loss + 0.1 * loss.item()

        # tqdm postfix (per-batch)
        postfix = {
            "loss(batch)": f"{loss.item():.4f}",
            "loss(ema)": f"{ema_loss:.4f}",
            "ppl(ema)": f"{math.exp(ema_loss):.2f}",
        }
        if grad_norm is not None:
            postfix["grad_norm"] = f"{grad_norm:.2f}"
        pbar.set_postfix(postfix)

        # prints périodiques optionnels
        if log_interval and (b + 1) % log_interval == 0:
            avg_loss_so_far = total_loss / max(total_tokens, 1)
            print(f"  step {b+1:05d} | {desc} avg_loss {avg_loss_so_far:.4f} | ppl {math.exp(avg_loss_so_far):.2f}")

    avg_loss = total_loss / max(total_tokens, 1)
    ppl = float(math.exp(avg_loss))
    elapsed = time.time() - start_time
    return avg_loss, ppl, elapsed

# -------------------------------------------------
# Entraînement
# -------------------------------------------------
EPOCHS = 10
best_val = float('inf')

print("===== Démarrage de l'entraînement =====")
print(f"device: {device} | vocab: {len(train_dataset.vocab)} | "
      f"batch_size: {getattr(train_loader, 'batch_size', 'NA')} | seq_len: {getattr(train_dataset, 'block_size', 'NA')}")
print(f"pin_memory(train_loader)={getattr(train_loader, 'pin_memory', 'NA')} | "
      f"num_workers={getattr(train_loader, 'num_workers', 'NA')}")

for epoch in range(1, EPOCHS + 1):
    print(f"\n----- Epoch {epoch:02d}/{EPOCHS} -----")
    train_loss, train_ppl, train_time = run_epoch(
        train_loader, train=True, desc=f"train[{epoch:02d}]", log_interval=100, vocab_size=len(train_dataset.vocab)
    )
    val_loss, val_ppl, val_time = run_epoch(
        test_loader, train=False, desc=f"eval [{epoch:02d}]", log_interval=None, vocab_size=len(train_dataset.vocab)
    )

    print(f"[{epoch:02d}] train loss {train_loss:.4f} | ppl {train_ppl:.2f} | time {train_time:.1f}s")
    print(f"[{epoch:02d}]   val loss {val_loss:.4f} | ppl {val_ppl:.2f} | time {val_time:.1f}s")

    # checkpoint si amélioration
    if val_loss < best_val:
        best_val = val_loss
        ckpt_path = f"best_decoder.pt"
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "metrics": {
                "train_loss": train_loss, "train_ppl": train_ppl,
                "val_loss": val_loss, "val_ppl": val_ppl,
            },
            "config": {
                "vocab_size": len(train_dataset.vocab),
                "embed_size": EMBED_SIZE,
                "num_layers": NUM_LAYERS,
                "forward_expansion": FORWARD_EXPANSION,
                "dropout": DROPOUT,
                "max_length": MAX_LENGTH,
                "block_size": BLOCK_SIZE,
            }
        }, ckpt_path)
        print(f"✓ checkpoint sauvegardé → {ckpt_path} (val_loss={val_loss:.4f})")
    else:
        print(f"↳ pas d'amélioration (best_val={best_val:.4f})")



MPS visible (Apple Silicon)
✓ model devices -> params: ['mps'] | buffers: none
Paramètres: 3.24M (dont entraînables: 3.24M)
===== Démarrage de l'entraînement =====
device: mps | vocab: 40 | batch_size: 64 | seq_len: 256
pin_memory(train_loader)=False | num_workers=0

----- Epoch 01/10 -----


train[01]:   0%|          | 0/781 [00:00<?, ?it/s]

✓ batch ok -> (64, 256) on mps:0 (expected mps) dtypes: x=torch.int64, y=torch.int64
✓ logits ok -> [B,T,V]=(64, 256, 40) on mps:0, dtype=torch.float32


train[01]:  13%|█▎        | 100/781 [01:13<07:56,  1.43it/s, loss(batch)=2.5042, loss(ema)=2.4969, ppl(ema)=12.14, grad_norm=0.21]

  step 00100 | train[01] avg_loss 2.6621 | ppl 14.33


...

train[10]:  51%|█████     | 400/781 [03:32<03:00,  2.11it/s, loss(batch)=0.9142, loss(ema)=0.8980, ppl(ema)=2.45, grad_norm=0.73]

  step 00400 | train[10] avg_loss 0.9175 | ppl 2.50


train[10]:  64%|██████▍   | 500/781 [04:22<02:15,  2.07it/s, loss(batch)=0.8976, loss(ema)=0.8965, ppl(ema)=2.45, grad_norm=0.84]

  step 00500 | train[10] avg_loss 0.9138 | ppl 2.49


KeyboardInterrupt: 